In [1]:
from catboost import CatBoostClassifier
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import GridSearchCV
import xlsxwriter
from random import randint
random_state = 7656
from preprocessing import stds, stats, removal, cv_preprocessing
from load_data import load_data
import os

Using TensorFlow backend.
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
class CustomAnovaFeatureSelection(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=None):
        super().__init__()

        self.n_features = n_features
        self.features = []
        
    def fit(self, X, y=None):
        if len(X.columns) < 3:
            return X
        
        if self.n_features is None:
            self.n_features = max(int(np.sqrt(len(X.columns))), 3)
        
        features_significance = {}
        # three way anova

        for i, a in enumerate(X.columns):
            for j, b in enumerate(X.columns[:i]):
                for c in X.columns[:j]:
                    
                    model = ols(f'y ~ {a} + {b} + {c} + {a}:{b} + {a}:{c} + {b}:{c} + {a}:{b}:{c}', data=X).fit()
                    mo = sm.stats.anova_lm(model,typ=3)
                    print(a, b, c)
                    a_significance = [mo['PR(>F)'][f'{a}'], mo['PR(>F)'][f'{a}:{c}'], mo['PR(>F)'][f'{a}:{b}'], mo['PR(>F)'][f'{a}:{b}:{c}']]
                    if f'{a}' in features_significance.keys():
                        a_significance.append(features_significance[f'{a}'])
                    features_significance[f'{a}'] = min(a_significance)
                    
                    b_significance = [mo['PR(>F)'][f'{b}'], mo['PR(>F)'][f'{b}:{c}'], mo['PR(>F)'][f'{a}:{b}'], mo['PR(>F)'][f'{a}:{b}:{c}']]
                    if f'{b}' in features_significance.keys():
                        b_significance.append(features_significance[f'{b}'])
                    features_significance[f'{b}'] = min(b_significance)
                    
                    c_significance = [mo['PR(>F)'][f'{c}'], mo['PR(>F)'][f'{a}:{c}'], mo['PR(>F)'][f'{b}:{c}'], mo['PR(>F)'][f'{a}:{b}:{c}']]
                    if f'{c}' in features_significance.keys():
                        c_significance.append(features_significance[f'{c}'])
                    features_significance[f'{c}'] = min(c_significance)                    
        
        features = sorted(X.columns, key=lambda x: features_significance[x])
        self.features = features[:self.n_features]
        
        #print(self.features)
        i = randint(start, end)
        kendall_feats = X.columns.insert(0, 'Features')
        with xlsxwriter.Workbook(f'CustomAnovaFeatureSelection_{i}.xlsx') as workbook:
            worksheet = workbook.add_worksheet()
            worksheet.write_column(0, 0, kendall_feats)
        
        return self

    def transform(self, X, y=None):
        
        return X[self.features]

In [3]:
def clean_features(X):
    rename_cols = {i: i.replace('.', '').replace(' ', '') for i in X.columns}
    return X.rename(rename_cols, axis=1)

In [4]:
df_preprocessed, features, target_feature = load_data()
X, X_out, Y, y_out = train_test_split(df_preprocessed[features], df_preprocessed[target_feature[0]],\
                                      test_size=0.25, random_state=random_state,\
                                      stratify=df_preprocessed[target_feature[0]])






T2


pcl1
PCL1_Broad
PCL1_Strict
(array([ 20, 502, 585, 688], dtype=int64),)


In [5]:

for i in [random_state, random_state-19, random_state+34, random_state-34, random_state+19]:
    X_train, X_test, y_train, y_test = train_test_split(X, Y,  random_state=i, test_size=0.2, stratify=Y)
    cv = StratifiedKFold(6, random_state=i, shuffle=True)

    X_train, X_test = cv_preprocessing(X_train, X_test, i)
    
    X_train, X_test = clean_features(X_train), clean_features(X_test)
    pipe = Pipeline(steps=[
    ('rfe', RFE(ExtraTreesClassifier(), 15)),
    ('classifier', CatBoostClassifier(verbose=0, random_state=i))])
        
    grid_params = [{
    'classifier__class_weights':[[1, 10]],#, [1, 15], [1, 30]],
    'classifier__l2_leaf_reg': [150],# 50],
    'classifier__depth': [6],#, 9]
    }]
    clf = GridSearchCV(pipe, grid_params, cv=cv, scoring='roc_auc')
    clf.fit(X_train, y_train.values.astype(int))#, fit_params = {'classifier__early_stopping_rounds':15})
    print(f"i = {i}, roc_auc = {clf.best_score_}, params = {clf.best_params_}")
    y_pred_target = clf.best_estimator_.predict_proba(X_test)[:, 1]
    print(f"holdout i = {i}, roc_auc = {roc_auc_score(y_test.astype(int), y_pred_target)}")
    for i, j in zip(X_train.columns, clf.best_estimator_['rfe'].ranking_):
        if j == 1:
            print(i)
    print("\n\n\n")

C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=15 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7656, roc_auc = 0.79845846524951, params = {'classifier__class_weights': [1, 10], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7656, roc_auc = 0.72
PCL1
phq2
PCL2
denial2
q68_AMNES_pcl1
q62_DREAM_pcl2
q69_DISINT_pcl2
q613_SLEEP_pcl2
q615_CONC_pcl2
binary_intrusion_pcl1
binary_avoidance_pcl1
delta_PCL
delta_q611_NUMB_pcl
std_pos_coping_t1
dffits_intrusion_pcl2






C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=15 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7637, roc_auc = 0.7806328207447609, params = {'classifier__class_weights': [1, 10], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7637, roc_auc = 0.825
T1Acc1t
phq2
PCL2
q62_DREAM_pcl1
q68_AMNES_pcl1
q611_NUMB_pcl1
q65_PHYS_pcl2
q67_AVSIT_pcl2
q69_DISINT_pcl2
q611_NUMB_pcl2
q613_SLEEP_pcl2
delta_emotional_support
delta_self_distraction
delta_q613_SLEEP_pcl
cooks_d_intrusion_pcl2






C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=15 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7690, roc_auc = 0.7340701794060003, params = {'classifier__class_weights': [1, 10], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7690, roc_auc = 0.79
phq2
PCL2
denial2
substance_use2
q68_AMNES_pcl1
q613_SLEEP_pcl1
q615_CONC_pcl1
q67_AVSIT_pcl2
q69_DISINT_pcl2
q611_NUMB_pcl2
delta_instrumental_support
delta_q617_STRTL_pcl
std_pos_coping_t2
cooks_d_PCL_score_pcl2
dffits_intrusion_pcl2






C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=15 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7622, roc_auc = 0.755031659882406, params = {'classifier__class_weights': [1, 10], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7622, roc_auc = 0.8066666666666666
phq1
phq2
PCL2
q65_PHYS_pcl1
q68_AMNES_pcl1
q617_STRTL_pcl1
q64_UPSET_pcl2
q615_CONC_pcl2
delta_PCL
delta_religion
delta_q68_AMNES_pcl
delta_q611_NUMB_pcl
delta_q613_SLEEP_pcl
delta_q617_STRTL_pcl
cooks_d_intrusion_pcl2






C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=15 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7675, roc_auc = 0.7937547112920247, params = {'classifier__class_weights': [1, 10], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7675, roc_auc = 0.81
phq1
PCL1
phq2
PCL2
positive_reframing1
COMT_Hap2_recode
q62_DREAM_pcl1
q68_AMNES_pcl1
q616_HYPER_pcl1
q65_PHYS_pcl2
q69_DISINT_pcl2
binary_intrusion_pcl1
delta_trait
delta_q617_STRTL_pcl
dffits_intrusion_pcl2




